In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/kaggle/input/santander-product-recommendation/train_ver2.csv.zip', low_memory = True)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
for x in data.columns:
    print('{}\n'.format(data[x].head()))

In [ ]:
data.info()

# EDA

In [ ]:
num_cols = [col for col in data.columns[:24] if data[col].dtype in ['int64','float64']]

data[num_cols].describe()

In [ ]:
num_cols = [i for i in data.columns[:24] if data[i].dtype in ['int64','float64']]

data[num_cols].describe()

In [ ]:
# need to fix

import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(data[num_cols])

In [ ]:
obj_cols = [col for col in data.columns[:24] if data[col].dtype in ["O"]]

data[obj_cols].describe()

In [ ]:
for x in obj_cols :
    uniq = np.unique(data[x].astype(str))
    print('-' * 50)
    print('# col {}, n_uniq {}, uniq {}'.format(x, len(uniq), uniq))

# EDA Data visualize

In [ ]:
skip_cols = ['ncodpers', 'renta']
for x in data.columns :
    if x in skip_cols:
        continue
        
        
    print('-' * 50)
    print('col : ', x)
    
    plt.figure(figsize=(20,15))
    sns.countplot(x=x, data=data, alpha=0.5)
    plt.show()

### variable 24: visualize (fin product)

In [ ]:
months = data['fecha_dato'].unique().tolist()

label_cols = data.columns[24:].tolist()

label_over_time = []
for i in range(len(label_cols)) :
    label_sum = data.groupby(['fecha_dato'])[label_cols[i]].agg('sum')
    label_over_time.append(label_sum.tolist())
    
    
label_sum_over_time = []
for i in range(len(label_cols)) :
    label_sum_over_time.append(np.asarray(label_over_time[i:]).sum(axis=0))
    
color_list = ['#F5B7B1','#D2B4DE','#AED6F1','#A2D9CE','#ABEBC6','#F9E79F','#F5CBA7','#CCD1D1']

plt.figure(figsize=(30,15))
for i in range(len(label_cols)) :
    sns.barplot(x=months, y=label_sum_over_time[i], color=color_list[i%8], alpha=0.7)
    
plt.legend([plt.Rectangle((0,0),1,1,fc=color_list[i%8], edgecolor='none')
           for i in range(len(label_cols))],
          label_cols, loc=1, ncol=2, prop={'size':16})

In [ ]:
label_sum_percent = (label_sum_over_time / (1.*np.asarray(label_sum_over_time).max(axis=0))) * 100

plt.figure(figsize=(30,15))
for x in range(len(label_cols)) :
    sns.barplot(x=months, y=label_sum_percent[x], color=color_list[x%8],alpha=0.7)
    
plt.legend([plt.Rectangle((0,0),1,1, fc=color_list[x%8], edgecolor='black')
           for x in range(len(label_cols))],
          label_cols, loc=1, ncol=2, prop={'size':16})

# practice 12/07 make a new value dataset

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('/kaggle/input/santander-product-recommendation/train_ver2.csv.zip', low_memory=True)

In [ ]:
train.head()

In [ ]:
products = train.columns[24:].tolist()

def data_to_int(str_date):
    Y, M, D = [int(x) for x in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

In [ ]:
# date to numberic and add to DF['int_date']
train['int_date'] = train['fecha_dato'].map(data_to_int).astype(np.int8)

train_copy = train.copy()
train_copy['int_date'] += 1
train_copy.columns = [col + '_prev' if col not in ['ncodpers', 'int_date']
                     else col for col in train.columns]

In [ ]:
df_train = train.merge(train_copy, on=['ncodpers','int_date'], how='left')